In [ ]:
# Начнём с загрузки всех нужных библиотек для обработки данных, которым уже была присвоена разметка по категориям: 
# 1. Нравится скорость отработки заявок
# 2. Нравится качество выполнения заявки
# 3. Нравится качество работы сотрудников
# 4. Понравилось выполнение заявки
# 5. Вопрос решен
# 6. Другое 
#
# PS: В данных есть комментарии которые обладают сразу несколькими категориями

In [1]:
import pandas as pd
import json

In [ ]:
# Удаляем ненужные столбцы: По моему мнению они не повлияют на точность модели и смогут визуально упростить работу с данными.

In [2]:
df = pd.read_csv('comments_procces.csv')

columns_to_drop = ['lead_time', 'updated_at', 'created_at', "annotator"]
df = df.drop(columns=columns_to_drop, errors='ignore')

display(df.head())

,annotation_id,comment,id,rating,taxonomy,Вопрос решен,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Нравится скорость отработки заявок,Понравилось выполнение заявки
0,101,спасибо,2945792,5,"[{""taxonomy"":[[""Вопрос решен""]]}]",NaN,NaN,NaN,NaN,NaN
1,102,спасибо!,3234340,5,"[{""taxonomy"":[[""Вопрос решен""]]}]",NaN,NaN,NaN,NaN,NaN
2,103,Отлично,3380332,5,"[{""taxonomy"":[[""Вопрос решен""]]}]",NaN,NaN,NaN,NaN,NaN
3,104,Благодарю за оперативное решение проблемы !,3381812,5,"[{""taxonomy"":[[""Нравится скорость отработки за...",NaN,NaN,NaN,NaN,NaN
4,105,Прекрасный специалист! Побольше таких,3461991,5,"[{""taxonomy"":[[""Нравится качество работы сотру...",NaN,NaN,NaN,NaN,NaN


In [ ]:
# Сделаем вывод общей информации о данных

In [3]:
print("Общая информация о данных:")
df.info()

Общая информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   annotation_id                         750 non-null    int64  
 1   comment                               750 non-null    object 
 2   id                                    750 non-null    int64  
 3   rating                                750 non-null    int64  
 4   taxonomy                              750 non-null    object 
 5   Вопрос решен                          0 non-null      float64
 6   Нравится качество выполнения заявки   0 non-null      float64
 7   Нравится качество работы сотрудников  0 non-null      float64
 8   Нравится скорость отработки заявок    0 non-null      float64
 9   Понравилось выполнение заявки         0 non-null      float64
dtypes: float64(5), int64(3), object(2)
memory usage: 58.7+ KB


In [ ]:
# Проведём обработку данных путём заполнения столбцов категорий
# Там где в столбце упоминается категория то происходит заполенение

In [4]:
categories = [
    "Нравится скорость отработки заявок",
    "Нравится качество выполнения заявки",
    "Нравится качество работы сотрудников",
    "Понравилось выполнение заявки",
    "Вопрос решен",
    "Другое"
]

for category in categories:
    if category not in df.columns:
        df[category] = 0

def extract_categories(taxonomy_str):
    try:
        data = json.loads(taxonomy_str)
        cats = []
        if isinstance(data, list):
            for item in data:
                if isinstance(item, dict) and 'taxonomy' in item:
                    nested = item['taxonomy']
                    if isinstance(nested, list):
                        for sublist in nested:
                            if isinstance(sublist, list):
                                cats.extend(sublist)
        return [cat.strip() for cat in cats]
    except:
        return []

for idx, row in df.iterrows():
    cats_in_cell = extract_categories(row['taxonomy'])
    for category in categories:
        df.at[idx, category] = 1 if category in cats_in_cell else 0

print("Обработка завершена.")

Обработка завершена.


In [ ]:
# Сделаем проверку проделанной работы.

In [5]:
df

,annotation_id,comment,id,rating,taxonomy,Вопрос решен,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Нравится скорость отработки заявок,Понравилось выполнение заявки,Другое
0,101,спасибо,2945792,5,"[{""taxonomy"":[[""Вопрос решен""]]}]",1.0,0.0,0.0,0.0,0.0,0
1,102,спасибо!,3234340,5,"[{""taxonomy"":[[""Вопрос решен""]]}]",1.0,0.0,0.0,0.0,0.0,0
2,103,Отлично,3380332,5,"[{""taxonomy"":[[""Вопрос решен""]]}]",1.0,0.0,0.0,0.0,0.0,0
3,104,Благодарю за оперативное решение проблемы !,3381812,5,"[{""taxonomy"":[[""Нравится скорость отработки за...",1.0,0.0,0.0,1.0,0.0,0
4,105,Прекрасный специалист! Побольше таких,3461991,5,"[{""taxonomy"":[[""Нравится качество работы сотру...",0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
745,746,Спасибо большое за оперативное реагирование и ...,3930202,5,"[{""taxonomy"":[[""Нравится скорость отработки за...",0.0,0.0,0.0,1.0,1.0,0
746,747,Большое спасибо за оперативность решения вопроса!,3930255,5,"[{""taxonomy"":[[""Нравится скорость отработки за...",0.0,0.0,0.0,1.0,0.0,0
747,748,Спасибо за оперативность.,3930372,5,"[{""taxonomy"":[[""Нравится скорость отработки за...",0.0,0.0,0.0,1.0,0.0,0
748,749,грамотный специалист.,3930387,5,"[{""taxonomy"":[[""Нравится качество работы сотру...",0.0,0.0,1.0,0.0,0.0,0


In [ ]:
# После проверки убедился что все столбцы были заполенены. 
# Столбец taxonomy потерял свою актуальность - удаляю его. 
# Ровно также дропаю дубликаты комментариев. 

In [6]:
df = df.drop_duplicates(subset=['comment'])

if 'taxonomy' in df.columns:
    df = df.drop(columns=['taxonomy'])

In [ ]:
# Проверяю как выглядит обработанные данные

In [10]:
df

,annotation_id,comment,id,rating,Вопрос решен,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Нравится скорость отработки заявок,Понравилось выполнение заявки,Другое
0,101,спасибо,2945792,5,1.0,0.0,0.0,0.0,0.0,0
1,102,спасибо!,3234340,5,1.0,0.0,0.0,0.0,0.0,0
2,103,Отлично,3380332,5,1.0,0.0,0.0,0.0,0.0,0
3,104,Благодарю за оперативное решение проблемы !,3381812,5,1.0,0.0,0.0,1.0,0.0,0
4,105,Прекрасный специалист! Побольше таких,3461991,5,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
598,745,Заменили,3930019,5,1.0,0.0,0.0,0.0,0.0,0
599,746,Спасибо большое за оперативное реагирование и ...,3930202,5,0.0,0.0,0.0,1.0,1.0,0
600,747,Большое спасибо за оперативность решения вопроса!,3930255,5,0.0,0.0,0.0,1.0,0.0,0
601,749,грамотный специалист.,3930387,5,0.0,0.0,1.0,0.0,0.0,0


In [ ]:
# Работа с данными проведена успешно!
# Сохраняю всё в отдельный файл.

In [7]:
df.to_csv('comments_processed_time.csv', index=False)

In [ ]:
# Решаю сделать статистику по каждому столбцу
# Статистика радует :) 

In [8]:
df = pd.read_csv('comments_processed_time.csv')

columns = [
    'Нравится скорость отработки заявок',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Понравилось выполнение заявки',
    'Вопрос решен',
    'Другое'
]

for col in columns:
    if col in df.columns:
        count_ones = (df[col] == 1).sum()
        print(f"Статистика по столбцу: {col}: {count_ones}")
    else:
        print(f"Столбец '{col}' не найден в файле.")

Статистика по столбцу: Нравится скорость отработки заявок: 230
Статистика по столбцу: Нравится качество выполнения заявки: 42
Статистика по столбцу: Нравится качество работы сотрудников: 144
Статистика по столбцу: Понравилось выполнение заявки: 102
Статистика по столбцу: Вопрос решен: 236
Статистика по столбцу: Другое: 35


In [ ]:
# Начинаю создавать и обучать модель.
# Работа с данными полностью закончена.